In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf 
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import  Counter
import nltk
nltk.download('stopwords')
import re

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.stats import norm, skew 
from scipy.special import boxcox1p
from sklearn.preprocessing import RobustScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/world-happiness-report-2021/world-happiness-report-2021.csv')
d1 = pd.read_csv('../input/world-happiness-report-2021/world-happiness-report.csv')

In [ ]:
data

In [ ]:
# Checking if there are any missing values
missing_value = data.isnull().sum().sort_values(ascending = False)
missing_perc = (data.isnull().sum()*100/data.shape[0]).sort_values(ascending = False)
value = pd.concat([missing_value,missing_perc],axis=1,keys=['Count','%'])
display(value.head(20).style.background_gradient(cmap = 'Reds', axis = 0))

**There are no missing values.**

In [ ]:
data.describe(include='all').transpose()

In [ ]:
# Checking number of unique values in each column
data.nunique()

In [ ]:
# Checking the histogram of the different columns
data.hist(figsize=(30,25))
plt.show()

We would expect the countries where **Positive attributes** are **high** to be much happier for eg: Social support, Life expectancy etc being on the higher side.



In [ ]:
fig = plt.figure(figsize=(30,15))
gs = fig.add_gridspec(3,3)
ax0 = fig.add_subplot(gs[0,0])
# ax1 = fig.add_subplot(gs[0,1])

sns.barplot(ax=ax0,data = data, x= 'Regional indicator',y='Ladder score',fill=True)
plt.xticks(rotation=90)
plt.show()
# sns.kdeplot(ax=ax1,data = data, x= 'Hardness',hue='Ladder ',fill=True)
# sns.kdeplot(ax=ax2,data = data, x= 'Solids',hue='Potability',fill=True)

* From the Bargraph above, **Western Europe and NorthAmerica** have the highest Ladder Score i.e **Happiness Index**
*                          **South Asia and Sub-Saharan Africa** have the lowest Ladder Score.

In [ ]:
plt.figure(figsize = (20,10))
df_corr = data.corr()
sns.heatmap(df_corr,fmt=".2f",annot=True,cmap='YlGnBu')

* As you may know, variables are highly correlated if they are closer to 1.
 
* There are various variables that have a high correlation.

In [ ]:
sub_df = ['Ladder score','Logged GDP per capita', 'Social support', 'Healthy life expectancy','Freedom to make life choices', 'Generosity','Perceptions of corruption', 'Ladder score in Dystopia']

sns.pairplot(data[sub_df],height=5)

Used Pairplot to describe relations between the different variables.

In [ ]:
vars = data[['Country name','Regional indicator','Logged GDP per capita','Social support','Ladder score',
               'Healthy life expectancy','Freedom to make life choices','Perceptions of corruption']].sort_values('Ladder score', ascending = False)

In [ ]:
var1 = vars.head(5)
fig = plt.figure(figsize=(30,50))
gs = fig.add_gridspec(3,3)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])

sns.barplot(ax=ax0, x = 'Logged GDP per capita',y = 'Country name',data = var1)
sns.barplot(ax=ax1,x = 'Social support',y = 'Country name',data = var1)
sns.barplot(ax=ax2,x = 'Ladder score',y = 'Country name',data = var1)
sns.barplot(ax=ax3,x = 'Healthy life expectancy',y = 'Country name',data = var1)
sns.barplot(ax=ax4,x = 'Freedom to make life choices',y = 'Country name',data = var1)
sns.barplot(ax=ax5,x = 'Perceptions of corruption',y = 'Country name',data = var1)

We can see the top 5 Happy countries and their stats across different variables.

In [ ]:
var1 = vars.tail(5)
fig = plt.figure(figsize=(30,50))
gs = fig.add_gridspec(3,3)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])

sns.barplot(ax=ax0, x = 'Logged GDP per capita',y = 'Country name',data = var1)
sns.barplot(ax=ax1,x = 'Social support',y = 'Country name',data = var1)
sns.barplot(ax=ax2,x = 'Ladder score',y = 'Country name',data = var1)
sns.barplot(ax=ax3,x = 'Healthy life expectancy',y = 'Country name',data = var1)
sns.barplot(ax=ax4,x = 'Freedom to make life choices',y = 'Country name',data = var1)
sns.barplot(ax=ax5,x = 'Perceptions of corruption',y = 'Country name',data = var1)

* The Countries with the least happiness index and their variables.
* Please notice the scale difference between the Top 5 and this. There is a huge difference.